## Health Insurance Prediction

### Build Regression Models

### Summary

This is **Part 2** of the project **Health Insurance Prediction: Build Regression Models**.

In this project I analyse <a href="https://https://www.kaggle.com/datasets/mirichoi0218/insurance">Medical Cost Personal</a>
dataset from <a href='https://www.kaggle.com/'>Kaggle</a>. 

The analysis covers **data visualization**, **feature engineering** and **building a linear regression model** to 
**predict insurance costs**.

It is primarily based in Python, using Numpy and Pandas for data manipulation, Matplotlib and Seaborn for visualizations and 
sklearn for building the machine leaning models.

### About Dataset

The **Insurance** dataset has **1338 observations and 7 attributes** and contains medical personal costs billed by a health 
insurance company. The data was extracted from online datasets available for the book "Machine Learning with R" by Brett Lantz.


#### Attribute information

- **age:** Age of primary beneficiary
- **sex:** Insurance contractor gender, female, male
- **bmi:** Body Mass Index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg/m^2) using the ratio of height to weight, ideally 18.5 to 24.9
- **children:** Number of children covered by health insurance / Number of dependents
- **smoker:** Smoking
- **region:** The beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- **charges:** Individual medical costs billed by health insurance

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Load data

In [2]:
# Retrieve data in a pandas dataframe
insurance = pd.read_csv('insurance.csv')

# Print the shape and the first rows
print(insurance.shape, "\n")
insurance.head()

(1338, 7) 



,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Get columns info

In [3]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


The dataset has four numerical and three categorical columns.

### Descriptive statistics

In [4]:
insurance.describe().style.format("{:.2f}")

,age,bmi,children,charges
count,1338.00,1338.00,1338.00,1338.00
mean,39.21,30.66,1.09,13270.42
std,14.05,6.10,1.21,12110.01
min,18.00,15.96,0.00,1121.87
25%,27.00,26.30,0.00,4740.29
50%,39.00,30.40,1.00,9382.03
75%,51.00,34.69,2.00,16639.91
max,64.00,53.13,5.00,63770.43


The beneficiaries’ gender and region are evenly distributed, having their age ranging from 18 to 64 years old. The bmi has a 
minim of 16 and a maximum of 54. Non-smokers outnumber smokers 4 to 1. 

The average medical cost is USD 13,270, higher than the median value of USD 9382, indicating a right skewed distribution.

In [5]:
# Make a copy of dataframe
insurance1 = insurance.copy()

## Data Preprocessing

For data preprocessing I followed the steps below.

### Remove duplicates

In [13]:
# Remove duplicates
insurance = insurance.drop_duplicates()
# Check that duplicates were removed
insurance.duplicated().value_counts()

False    1337
dtype: int64

### Check for missing values

In [14]:
insurance.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

The dataset has **no missing values**.

### Detect and Remove Outliers

We'll use **z-scores to detect the outliers**, since the **histogram of 'bmi'** has a **relatively normal distribution**.

In [15]:
# Getting z-scores from our data
zscores = zscore(insurance.loc[:, 'bmi'])

# Identifying Outliers with Z-scores
threshold = 3
outliers = insurance[np.abs(zscores) > threshold]

# Removing Outliers with Z-scores
non_outliers = insurance[np.abs(zscores) <= threshold]

print(f'Data before removing outliers: {len(insurance)}')
print(f'Data after removing outliers: {len(non_outliers)}')

Data before removing outliers: 1337
Data after removing outliers: 1333


### Binning 'age' variable

In [16]:
# Create custom bins
bins_age = [17, 25, 40, 50, 65]
# Label the bins
age_categ = ['17-25', '25-40', '40-50', '50-65'] 
# Bin age variable
binAge = pd.cut(insurance.loc[:, 'age'], bins_age, labels=age_categ)      
insurance['age'] = binAge

### One-hot Encoding categorical variables

In [17]:
# One-hot encoding categorical variables
features = pd.get_dummies(insurance, columns=['age', 'children', 'region'])
# Drop first values in columns sex and smoker
features = pd.get_dummies(features, columns=['sex', 'smoker'], drop_first=True)

print(features.shape)

(1337, 18)


### Normalize features

In [18]:
# Select columns to be normalized
cols_normalize = ['bmi', 'charges']

# Create a Numpy array with the values of columns
x = features[cols_normalize].values

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
x_scaled = scaler.fit_transform(x)

# Create a dataframe for normalized features
features_temp = pd.DataFrame(x_scaled, 
                columns=cols_normalize, 
                index=features.index)

# Replace columns in features dataframe 
# with normalized values
features[cols_normalize] = features_temp

## Build  Regression Models

We'll build the following regression models to predict insurance costs:
- **Linear Regression**
- **Random Forest**
- **Polynomial Regression**

### Split the dataset in train and test data

In [19]:
# Create the label
y = features['charges']

# Drop the column "charges"
features = features.drop('charges', axis=1)

# Make a list with all the columns in dataframe
cols = list(features.columns)

# Create the features dataframe
X = features[cols]

# Split the dataset into training 
# set (70%) and testing set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                        test_size=0.3, random_state=123)

# Print the shape of the training and testing datasets
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

X_train:  (935, 17)
y_train:  (935,)
X_test:  (402, 17)
y_test:  (402,)


### Linear Regression

In [20]:
# Fit a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions
y_pred = lr.predict(X_test)

# Print the intercept and 
# coeficients
print("Intercept:", "\n", lr.intercept_)
print("Coefficients:", "\n", lr.coef_)
print()


# Evaluate the model
mae = mean_absolute_error(y_true=y_test, 
                          y_pred=y_pred) 
mse = mean_squared_error(y_true=y_test,
                         y_pred=y_pred)                  
rmse = mean_squared_error(y_true=y_test,
                          y_pred=y_pred,
                          squared=False) 
r2 = r2_score(y_true=y_test,
              y_pred=y_pred)

print("Linear Regression metrics:")
print("MAE: %.3f" % mae) 
print("MSE: %.3f" % mse) 
print("RMSE: %.3f" % rmse)
print("R-squared: %.3f" % r2)

Intercept: 
 1698521079710.9395
Coefficients: 
 [ 2.02744489e-01 -8.45226500e+11 -8.45226500e+11 -8.45226500e+11
 -8.45226500e+11 -3.00895247e+11 -3.00895247e+11 -3.00895247e+11
 -3.00895247e+11 -3.00895247e+11 -3.00895247e+11 -5.52399333e+11
 -5.52399333e+11 -5.52399333e+11 -5.52399333e+11 -4.60711042e-03
  3.80042541e-01]

Linear Regression metrics:
MAE: 0.066
MSE: 0.009
RMSE: 0.092
R-squared: 0.769


The **R2 score** for **Linear Regression** has a good value of **0.77**.

### Random Forest

In [21]:
# Fit a random forest model
forest = RandomForestRegressor(n_estimators = 100,
                              criterion = 'squared_error',
                              random_state = 1,
                              n_jobs = -1)
forest.fit(X_train, y_train)

# Make predictions
y_pred = forest.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_true=y_test, 
                          y_pred=y_pred) 
mse = mean_squared_error(y_true=y_test,
                         y_pred=y_pred)                  
rmse = mean_squared_error(y_true=y_test,
                          y_pred=y_pred,
                          squared=False) 
r2 = r2_score(y_true=y_test,
              y_pred=y_pred)

print("Random Forest metrics:")
print("MAE: %.3f" % mae) 
print("MSE: %.3f" % mse) 
print("RMSE: %.3f" % rmse)
print("R-squared: %.3f" % r2)

Random Forest metrics:
MAE: 0.045
MSE: 0.006
RMSE: 0.074
R-squared: 0.850


**Random forest** model has a **higher score** than Linear Regression of **0.85**.

### Polynomial Regression

In [22]:
# Map categories in smpker
insurance1['smoker'] = insurance1['smoker'].map({'yes':0, 'no':1})

X = insurance1.drop(['charges', 'sex', 'region'], axis = 1)
y = insurance1.charges

pol = PolynomialFeatures (degree = 2)
X_pol = pol.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_pol, y, test_size=0.2, random_state=0)

# Fit a polynomial regression model
Pol_reg = LinearRegression()
Pol_reg.fit(X_train, y_train)

# Make predictions
y_pred = Pol_reg.predict(X_test)

# Print the intercept and 
# coeficients
print("Intercept:", "\n", Pol_reg.intercept_)
print("Coefficients:", "\n", Pol_reg.coef_)
print()

# Evaluate the model
mae = mean_absolute_error(y_true=y_test, 
                          y_pred=y_pred) 
mse = mean_squared_error(y_true=y_test,
                          y_pred=y_pred)                  
rmse = mean_squared_error(y_true=y_test, 
                          y_pred=y_pred, 
                          squared=False) 
r2 = r2_score(y_true=y_test, 
              y_pred=y_pred)

print("Polynomial Regression metrics")
print("MAE: %.3f" % mae) 
print("MSE: %.3f" % mse) 
print("RMSE: %.3f" % rmse)
print("R-squared: %.3f" % r2)

Intercept: 
 -24379.8511270221
Coefficients: 
 [ 0.00000000e+00 -3.59521612e+01  1.93210872e+03  4.05873171e+02
  9.52698471e+03  3.04430186e+00  1.84508369e+00  6.01720286e+00
 -4.20849790e+00 -9.38983382e+00  3.81612289e+00 -1.40840670e+03
 -1.45982790e+02  4.46151855e+02  9.52698471e+03]

Polynomial Regression metrics
MAE: 2824.495
MSE: 18895160.099
RMSE: 4346.856
R-squared: 0.881


**Polynomial Regression** has an R2 score of **0.88**.

This is the **best score** from **all models**, so **Polynomial Regression** is the **best model** for **predicting insurance 
costs**.

If we consider the **Linear Regression**, as a **baseline model** (R2=0.77) and **Polynomial Regression**, as the **best model**
(R2=0.88), we can calculate the forecast accuracy improvement percentage, using the MAE for both models. The result shows 
a 33% improvement.

We can conclude that **Polynomial Regression** improved the **insurance forecast accuracy by 33%**, **increasing the R2 score 
from 0.77 to 0.88**.

### Conclusions

**Data visualization** allowed exploring the relationships between health insurance costs and several factors like age, 
health status (body mass index, smoking), gender, family size, and region. 
The main takeaway of the analysis is that **smoking has the highest impact on insurance charges**. Costs also **increase with 
BMI and age**. Insurance charges get **higher also for non-smokers** depending on **age, BMI**, and family size. 

Based on the findings from data exploration, I built and evaluated several regression models for predicting the insurance costs. 
**Polynomial Regression** achieved the **highest R2 score**, predicting the best patient's health insurance costs.

Comparing the Linear Regression and Polynomial Regression models, it can be concluded that **Polynomial Regression** improved 
the **insurance forecast accuracy by 33%**, **increasing the R2 score from 0.77 to 0.88**.